In [1]:
from tool import *
import plotly.graph_objects as go

In [2]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    sp = pickle.load(f)

# 1. 신규 생산계획 불러와 데이터 가공

In [3]:
pp = pd.read_excel(get_filename(), sheet_name=0, skiprows=1)

In [4]:
pp_date = []
for item in pp.columns:
    if type(item) == datetime.datetime:
        pp_date.append(item)

pp = pd.concat([pp.iloc[:, :7], pp[pp_date]], axis=1)
pp.columns = ['Target Week', 'ODF', 'TCL BOM', 'drop3', 'drop4', 'Production Date', 'WO'] + pp_date
pp = pp[['Target Week', 'ODF', 'TCL BOM', 'Production Date', 'WO'] + pp_date]
pp = pp.set_index(['Target Week', 'ODF', 'TCL BOM'])[pp_date].stack().reset_index()
pp.columns = ['Target Week', 'ODF', 'TCL BOM', 'PP Date', 'QTY']
pp = pp[pp['TCL BOM'].notnull()].reset_index(drop=True)
pp

,Target Week,ODF,TCL BOM,PP Date,QTY
0,24/01/29W,DOK7518033,X3-22M205M3-COS1AU,2024-01-22,840.0
1,24/01/29W,DOK7518036,X3-22M205M3-VMS1AU,2024-01-22,1260.0
2,24/02/05W,DOK7518052,X3-22M205M3-HKS0AU,2024-01-23,280.0
3,24/01/29W,DOK7518032,X3-27M205M3-USS0AU,2024-01-31,3840.0
4,24/02/05W,DOK7518132,X3-22M205M3-AAS1AU,2024-01-27,252.0
...,...,...,...,...,...
103,24/03/18W,NaN,X3-27M205M3-HKS0AU,2024-03-08,128.0
104,24/03/18W,NaN,X3-27M205M3-INS0AU,2024-03-08,512.0
105,24/03/18W,NaN,X3-27M205M3-USS0DU,2024-03-08,512.0
106,24/03/18W,NaN,X3-27M205M3-USS0EU,2024-03-08,2560.0


In [5]:
pp['QTY'].sum()

117885.0

In [6]:
pp = pp[pp['QTY'].apply(lambda x:type(x) in [int, float])]
pp['QTY'] = pp['QTY'].astype(int)
pp['Rep PMS'] = pp['TCL BOM'].map(tcl_bom_map)

In [7]:
pp[pp['Rep PMS'].isnull()] # Rep PMS 명의 결측치 확인

,Target Week,ODF,TCL BOM,PP Date,QTY,Rep PMS


In [8]:
pp['Mapping Model.Suffix'] = pp['Rep PMS'].map(make_dict_from_df(sp[['Rep PMS', 'Mapping Model.Suffix']].drop_duplicates())).fillna('TBD')

In [9]:
pp[pp['Mapping Model.Suffix'].isnull()] # Mapping Model 명의 결측치 확인

,Target Week,ODF,TCL BOM,PP Date,QTY,Rep PMS,Mapping Model.Suffix


In [10]:
pp['Updated_at'] = datetime.datetime(2024,1,30,0,27) # 업데이트 일자

In [11]:
c1 = pp['TCL BOM'] == 'X3-27M205M3-USS0EU' # Costco 2 pack 수량을 outer box 기준으로 재조정

In [12]:
pp[c1] 

,Target Week,ODF,TCL BOM,PP Date,QTY,Rep PMS,Mapping Model.Suffix,Updated_at
40,24/02/19W,DOK7518198,X3-27M205M3-USS0EU,2024-02-05,1024,27MC41D-BS.CCCRMV,27MC41D-BS.ACCQ,2024-01-30 00:27:00
106,24/03/18W,NaN,X3-27M205M3-USS0EU,2024-03-08,2560,27MC41D-BS.CCCRMV,27MC41D-BS.ACCQ,2024-01-30 00:27:00


In [13]:
pp.loc[c1, 'QTY'] = pp.loc[c1, 'QTY'] / 2

In [14]:
pp.shape

(108, 8)

In [15]:
with open('D:/Data/TCL PP_result.bin', 'rb') as f:
    ppr_db = pickle.load(f)

In [16]:
ppr_db['PP Date'].max()

Timestamp('2024-01-20 00:00:00')

In [17]:
c1 = pp['PP Date'] >= '2024-01-22'
c2 = pp['PP Date'] <= '2024-01-28'

In [18]:
pp[c1 & c2]

,Target Week,ODF,TCL BOM,PP Date,QTY,Rep PMS,Mapping Model.Suffix,Updated_at
0,24/01/29W,DOK7518033,X3-22M205M3-COS1AU,2024-01-22,840,22MR410-BB.CWPRMV,22MR410-B.AWPQ,2024-01-30 00:27:00
1,24/01/29W,DOK7518036,X3-22M205M3-VMS1AU,2024-01-22,1260,22MR410-BB.CTVRMV,22MR410-B.ATVQ,2024-01-30 00:27:00
2,24/02/05W,DOK7518052,X3-22M205M3-HKS0AU,2024-01-23,280,22MR410-BB.CHKRMV,22MR410-B.AHKQ,2024-01-30 00:27:00
4,24/02/05W,DOK7518132,X3-22M205M3-AAS1AU,2024-01-27,252,22MR410-BB.CAURMV,22MR410-B.AAUQ,2024-01-30 00:27:00
5,24/02/05W,DOK7518046,X3-22M205M3-ISS0AU,2024-01-28,840,22MR410-BB.CMFRMV,22MR410-B.AMFQ,2024-01-30 00:27:00
6,24/02/05W,DOK7518133,X3-22M205M3-THS1AU,2024-01-28,2184,22MR410-BB.CTMRMV,22MR410-B.ATMQ,2024-01-30 00:27:00
14,24/02/05W,DOK7518047,X3-24M205M3-ISS0AU,2024-01-28,576,24MR400-BA.CMFRMV,24MR400-B.AMFQ,2024-01-30 00:27:00
18,24/02/05W,DOK7518041,X3-27M205M3-INS0AU,2024-01-27,512,27MR400-BB.CTRRMV,27MR400-B.ATRQ,2024-01-30 00:27:00
19,24/02/05W,DOK7518054,X3-27M205M3-HKS0AU,2024-01-27,192,27MR400-BB.CHKRMV,27MR400-B.AHKQ,2024-01-30 00:27:00
20,24/02/05W,DOK7518042,X3-27M205M3-KRS0AU,2024-01-28,1280,27MR400-BB.CKRRMV,27MR400.AKRQ,2024-01-30 00:27:00


In [19]:
ppr_db = pd.concat([ppr_db, pp[c1 & c2]]).reset_index(drop=True)

In [20]:
with open('D:Data/TCL PP_result.bin', 'wb') as f: # PP reulst에 저장
    pickle.dump(ppr_db, f)

In [21]:
pp.shape

(108, 8)

In [22]:
pp = pp[pp['PP Date'] >= '2024-01-29'].reset_index(drop=True)

In [23]:
pp.shape

(96, 8)

In [24]:
with open('D:/Data/TCL PP_new.bin', 'rb') as f:
    pp_db = pickle.load(f)

In [25]:
pp_db

,Target Week,Country,ODF,TCL BOM,Mapping Model.Suffix,Rep PMS,Production Date,WO,PP Date,QTY,Updated_at
0,NaN,NaN,DOK7516345,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,NaT,2400.0,2023-10-19,2200,2023-10-17 00:36:00
1,NaN,NaN,DOK7516345,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,NaT,2400.0,2023-10-20,200,2023-10-17 00:36:00
2,NaN,NaN,DOK7516364,X3-24M205M3-ARS0AU,24MR400-B.AMIQ,24MR400-BA.CMIRMV,NaT,1440.0,2023-10-20,1440,2023-10-17 00:36:00
3,NaN,NaN,DOK7516348,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,NaT,2400.0,2023-10-20,400,2023-10-17 00:36:00
4,NaN,NaN,DOK7516348,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,NaT,2400.0,2023-10-21,2000,2023-10-17 00:36:00
...,...,...,...,...,...,...,...,...,...,...,...
1805,24/03/18W,NaN,NaN,X3-27M205M3-EUS0AU,27MR400-B.AEUQ,27MR400-BB.CEURMV,NaT,NaN,2024-03-12,512,2024-01-22 21:50:00
1806,24/03/18W,NaN,NaN,X3-27M205M3-HKS0AU,27MR400-B.AHKQ,27MR400-BB.CHKRMV,NaT,NaN,2024-03-12,128,2024-01-22 21:50:00
1807,24/03/18W,NaN,NaN,X3-27M205M3-INS0AU,27MR400-B.ATRQ,27MR400-BB.CTRRMV,NaT,NaN,2024-03-12,512,2024-01-22 21:50:00
1808,24/03/18W,NaN,NaN,X3-27M205M3-JNS0AU,27MR400-B.AJLQ,27MR400-BB.CJLRMV,NaT,NaN,2024-03-12,512,2024-01-22 21:50:00


In [26]:
pp_db = pd.concat([pp_db, pp]).reset_index(drop=True)

In [27]:
pp_db.shape

(1906, 11)

In [28]:
sorted(pp_db[pp_db['Updated_at'] == pp_db['Updated_at'].max()]['PP Date'].unique()) # 신규 PP의 생산일자가 맞게 되었는 지 확인

[numpy.datetime64('2024-01-29T00:00:00.000000000'),
 numpy.datetime64('2024-01-30T00:00:00.000000000'),
 numpy.datetime64('2024-01-31T00:00:00.000000000'),
 numpy.datetime64('2024-02-01T00:00:00.000000000'),
 numpy.datetime64('2024-02-02T00:00:00.000000000'),
 numpy.datetime64('2024-02-03T00:00:00.000000000'),
 numpy.datetime64('2024-02-04T00:00:00.000000000'),
 numpy.datetime64('2024-02-05T00:00:00.000000000'),
 numpy.datetime64('2024-02-15T00:00:00.000000000'),
 numpy.datetime64('2024-02-16T00:00:00.000000000'),
 numpy.datetime64('2024-02-17T00:00:00.000000000'),
 numpy.datetime64('2024-02-18T00:00:00.000000000'),
 numpy.datetime64('2024-02-19T00:00:00.000000000'),
 numpy.datetime64('2024-02-20T00:00:00.000000000'),
 numpy.datetime64('2024-02-21T00:00:00.000000000'),
 numpy.datetime64('2024-02-23T00:00:00.000000000'),
 numpy.datetime64('2024-02-26T00:00:00.000000000'),
 numpy.datetime64('2024-02-27T00:00:00.000000000'),
 numpy.datetime64('2024-02-28T00:00:00.000000000'),
 numpy.datet

In [29]:
# 업데이트된 pp_db를 저장
with open('D:/Data/TCL PP_new.bin', 'wb') as f:
    pickle.dump(pp_db, f)